In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/home/alejandro/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import os
workers = 0 if os.name == 'nt' else 4
import numpy as np
import cv2
from PIL import Image, ImageDraw
from IPython import display
import matplotlib.patches as patches



In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

Running on device: cuda:0


In [4]:
# Create face detector
mtcnn = MTCNN(image_size=128,margin=0,select_largest=False, device=device)
help(MTCNN)

Help on class MTCNN in module facenet_pytorch.models.mtcnn:

class MTCNN(torch.nn.modules.module.Module)
 |  MTCNN(image_size=160, margin=0, min_face_size=20, thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=True, selection_method=None, keep_all=False, device=None)
 |  
 |  MTCNN face detection module.
 |  
 |  This class loads pretrained P-, R-, and O-nets and returns images cropped to include the face
 |  only, given raw input images of one of the following types:
 |      - PIL image or list of PIL images
 |      - numpy.ndarray (uint8) representing either a single image (3D) or a batch of images (4D).
 |  Cropped faces can optionally be saved to file
 |  also.
 |  
 |  Keyword Arguments:
 |      image_size {int} -- Output image size in pixels. The image will be square. (default: {160})
 |      margin {int} -- Margin to add to bounding box, in terms of pixels in the final image. 
 |          Note that the application of the margin differs slightly from the 

In [5]:
import re
numbers = re.compile(r'(\d+)')

def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)  # pause a bit so that plots are updated

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def readLandmarks(landmarks_dir):
        os.chdir(landmarks_dir)
        # iteramos en los archivos
        with open('points_descr.info') as f:
            col_names=f.readlines()
            df=pd.DataFrame(columns=col_names)

        for file in sorted(os.listdir(), key=numericalSort):
            # Check whether file is in text format or not
            if file.endswith(".jpg") or file.endswith(".JPG"):
                image_file=file
            if file.endswith(".pts"):
                with open(file) as f:
                    lines=f.readlines()
                    
                count=0
                dataframe_row=[]
                for line in lines: 
                    count +=1
                    if(count >= 4 and count<=33):
                        vector=[int(i) for i in line.split()]
                        dataframe_row.append(vector)
                dataframe_row.append(image_file)
                df.loc[len(df)] = dataframe_row
        os.chdir("..")

        return df

                
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = readLandmarks(root_dir)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 30])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 0:30]
        landmarks = np.array(landmarks)
        landmarks = np.array([np.array(x) for x in landmarks])
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    
            
            

In [ ]:

face_dataset=FaceLandmarksDataset('./FORENSE_AM')

fig = plt.figure()


for i in range(len(face_dataset)):
    sample = face_dataset[i+54]

    print(i, sample['image'].shape, sample['landmarks'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break


In [ ]:

face=mtcnn(face_dataset[0]['image'])
np_arr = face.cpu().detach().numpy()
np_arr=np_arr.transpose((1,2,0))
plt.imshow(np_arr)


In [45]:
import re
numbers = re.compile(r'(\d+)')

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def readLandmarks2(landmarks_dir,alpha):
        os.chdir(landmarks_dir)
        fnames=[]
        ids=[]
        masks=[]
        landmarks_list=[]
        face_x=[]
        face_y=[]
        face_w=[]
        face_h=[]
        
        # iteramos en los archivos y rellenamos los 4 primeros
        i=0
        for file in sorted(os.listdir(), key=numericalSort):
            # Check whether file is in text format or not
            if file.endswith(".jpg") or file.endswith(".JPG"):
                image_file=file
                fnames.append(image_file)
                ids.append(i)
                # Construimos las BB
                image = io.imread(image_file)
                if(len(image.shape)<3):
                    #Grey scale to RGB
                    image=np.array([image,image,image])
                    image=np.moveaxis(image, 0, -1)
                bb,prob=mtcnn.detect(image)
                if i==55:
                    face_x.append(bb[1,0])
                    face_y.append(bb[1,1]-5)
                    face_w.append(abs(bb[1,0]-bb[1,2]))
                    face_h.append(abs(bb[1,1]-bb[1,3])*alpha)
                elif i==57:
                    face_x.append(bb[2,0])
                    face_y.append(bb[2,1]-5)
                    face_w.append(abs(bb[2,0]-bb[2,2]))
                    face_h.append(abs(bb[2,1]-bb[2,3])*alpha)
                else:
                    face_x.append(bb[0,0])
                    face_y.append(bb[0,1]-5)
                    face_w.append(abs(bb[0,0]-bb[0,2]))
                    face_h.append(abs(bb[0,1]-bb[0,3])*alpha)
                i+=1

            if file.endswith(".pts"):
                with open(file) as f:
                    lines=f.readlines()
                    
                count=0
                landmarks=[]
                mask=[]
                for line in lines: 
                    count +=1
                    if(count >= 4 and count<=33):
                        vector=[int(i) for i in line.split()]
                        if(vector[0]==-1):
                            mask.append(0)
                        else:
                            mask.append(1)
                        landmarks.append(vector)
                
                landmarks_list.append(landmarks)
                masks.append(mask)
        
        df=pd.DataFrame({
            'fnames':fnames,
            'ra':ids,
            'landmarks_full':landmarks_list,
            'masks': masks,
            'face_x': face_x,
            'face_y': face_y,
            'face_w': face_w,
            'face_h': face_h
        })
        print(df)
        os.chdir("..")
        return df
    
class FaceLandmarksDataset2(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,root_dir, alpha, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = readLandmarks2(root_dir,alpha)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx,0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 2]
        landmarks = np.array(landmarks)
        landmarks = np.array([np.array(x) for x in landmarks])
        bb=[self.landmarks_frame.iloc[idx,4],self.landmarks_frame.iloc[idx,5],self.landmarks_frame.iloc[idx,6],self.landmarks_frame.iloc[idx,7]]
        sample = {'fname': image, 'landmarks': landmarks, 'bb':bb}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    

In [51]:
face_dataset=FaceLandmarksDataset2('./FORENSE_AM',1.1)
face_dataset.landmarks_frame.to_csv('./FORENSE_AM/annotations.csv')

        fnames   ra                                     landmarks_full  \
0        1.jpg    0  [[348, 394], [351, 389], [351, 375], [-1, -1],...   
1      2_1.jpg    1  [[-1, -1], [384, 924], [385, 884], [-1, -1], [...   
2      2_2.jpg    2  [[-1, -1], [737, 868], [750, 826], [-1, -1], [...   
3      3_1.jpg    3  [[619, 927], [615, 926], [615, 893], [-1, -1],...   
4      3_2.jpg    4  [[256, 979], [232, 958], [218, 930], [-1, -1],...   
..         ...  ...                                                ...   
159  157_2.jpg  159  [[-1, -1], [714, 569], [712, 554], [-1, -1], [...   
160    158.jpg  160  [[402, 285], [405, 283], [405, 272], [-1, -1],...   
161    159.jpg  161  [[243, 566], [247, 560], [245, 537], [-1, -1],...   
162    160.jpg  162  [[-1, -1], [314, 572], [310, 552], [-1, -1], [...   
163    163.jpg  163  [[-1, -1], [481, 695], [481, 680], [-1, -1], [...   

                                                 masks      face_x  \
0    [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,

In [ ]:
#REndimeinto del detector de caras en el dataset 
for i in range(164):
    image=face_dataset[i]['fname']
    fig,ax=plt.subplots()
    ax.imshow(image)
    rect=patches.Rectangle((face_dataset[i]['bb'][0],face_dataset[i]['bb'][1]),face_dataset[i]['bb'][2],face_dataset[i]['bb'][3], linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    ax.set_title('Sample #{}'.format(i))

    plt.show
    
